# Labeling Convective Mode
This notebook is used in labeling storm objects by their corresponding convective mode. This data will be used to assign a label convective objects for the purpose of using them for machine learning techniques. This is used as more of an example and introduction to machine learning as opposed to being a sound methodology for the automation of identfying storm mode (although wouldn't it be great if it did work out). Furthermore, this notebook should provide some fancy tricks in terms of using widgets in python.

In [1]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import cartopy
from sklearn.cluster import KMeans

import ipywidgets as widgets
from IPython.display import display, clear_output
from collections import Counter

import xarray as xr

%matplotlib inline

## Obtaining Data

In [2]:
#Open the pickle file.
pkl_data = pd.read_pickle('/home/scratch/jcorner1/Storm_Objects/mrms/mrms_storm_ids3.pkl')
pkl_data = pkl_data[(pkl_data['max_intensity'] >= 50.0)]
pkl_data

,uniq_id,area,convex_area,eccentricity,mean_intensity,max_intensity,intensity_variance,major_axis_length,minor_axis_length,solidity,...,xmax,ymin,ymax,coords,geometry,start_year,start_month,start_day,start_hour,start_time
30,mrms20200603r30,466.0,675.0,0.978882,32.110515,50.0,61.988323,58.994555,12.060162,0.690370,...,5093,481,518,"[[481, 5084], [481, 5085], [481, 5088], [481, ...","POLYGON ((514.000 5049.000, 511.000 5050.000, ...",2020,6,3,0,2020-06-03
34,mrms20200603r34,405.0,572.0,0.961438,34.646914,50.5,62.229651,45.848487,12.609321,0.708042,...,5040,510,544,"[[510, 5036], [511, 5034], [511, 5035], [511, ...","POLYGON ((541.000 5005.000, 535.000 5006.000, ...",2020,6,3,0,2020-06-03
41,mrms20200603r41,12289.0,19589.0,0.872354,28.635202,52.0,35.768501,206.574584,100.989224,0.627342,...,4880,550,686,"[[550, 4797], [551, 4782], [551, 4783], [551, ...","POLYGON ((662.000 4663.000, 570.000 4706.000, ...",2020,6,3,0,2020-06-03
49,mrms20200603r49,220.0,267.0,0.896060,37.415909,52.0,71.003156,26.010414,11.546880,0.823970,...,3215,572,585,"[[572, 3203], [572, 3204], [572, 3205], [572, ...","POLYGON ((577.000 3189.000, 575.000 3191.000, ...",2020,6,3,0,2020-06-03
64,mrms20200603r64,541.0,931.0,0.858435,29.041590,51.0,44.894296,44.832588,22.995672,0.581096,...,4849,656,681,"[[656, 4840], [657, 4838], [657, 4839], [657, ...","POLYGON ((668.000 4805.000, 662.000 4807.000, ...",2020,6,3,0,2020-06-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16050,mrms20200811r16050,763.0,1310.0,0.968391,35.617300,55.0,86.274249,74.931779,18.690692,0.582443,...,4642,2565,2595,"[[2565, 4569], [2565, 4570], [2565, 4572], [25...","POLYGON ((2566.000 4566.000, 2565.000 4569.000...",2020,8,11,0,2020-08-11
16056,mrms20200811r16056,492.0,839.0,0.707963,33.235772,57.0,75.234046,35.760524,25.255866,0.586412,...,4650,2597,2628,"[[2597, 4631], [2598, 4621], [2598, 4626], [25...","POLYGON ((2605.000 4613.000, 2600.000 4616.000...",2020,8,11,0,2020-08-11
16057,mrms20200811r16057,12190.0,23703.0,0.882392,28.823708,60.0,55.237912,245.754675,115.631289,0.514281,...,4844,2601,2793,"[[2601, 4693], [2601, 4694], [2601, 4695], [26...","POLYGON ((2654.000 4640.000, 2651.000 4642.000...",2020,8,11,0,2020-08-11
16058,mrms20200811r16058,205.0,284.0,0.951414,31.675610,54.0,66.200869,31.062994,9.564771,0.721831,...,4664,2630,2641,"[[2630, 4653], [2630, 4654], [2631, 4648], [26...","POLYGON ((2635.000 4632.000, 2633.000 4637.000...",2020,8,11,0,2020-08-11


## Creating Buttons and Actions

In [3]:
#Create a dropdown menu for classifying storm mode.
drop = widgets.Dropdown(
    options = ['Isolated Cell', 'Cluster of Cells', 'Supercell', 'Linear', 'Disorganized', ' '],
    value = ' ',
    description = 'Storm Mode:',
    disabled = False,)


In [4]:
#reset the values if needed!
row = 0
total = 0

In [5]:
#Long list provided to use with Iowa State MRMS data.
import matplotlib as mpl

def IS_radar_colormap():
    
    
    nws_reflectivity_colors = [
        '#000000', #-32
        '#000000', #-31.5
        '#000000', #-31
        '#000000', #-30.5
        '#000000', #-30
        '#000000', #-29.5
        '#000000', #-29
        '#000000', #-28.5
        '#000000', #-28
        '#000000', #-27.5
        '#000000', #-27
        '#000000', #-26.5
        '#000000', #-26
        '#000000', #-25.5
        '#000000', #-25
        '#000000', #-24.5
        '#000000', #-24
        '#000000', #-23.5
        '#000000', #-23
        '#000000', #-22.5
        '#000000', #-22
        '#000000', #-21.5
        '#000000', #-21
        '#000000', #-20.5
        '#000000', #-20
        '#000000', #-19.5
        '#000000', #-19
        '#000000', #-18.5
        '#000000', #-18
        '#000000', #-17.5
        '#000000', #-17
        '#000000', #-16.5
        '#000000', #-16
        '#000000', #-15.5
        '#000000', #-15
        '#000000', #-14.5
        '#000000', #-14
        '#000000', #-13.5
        '#000000', #-13
        '#000000', #-12.5
        '#000000', #-12
        '#000000', #-11.5
        '#000000', #-11
        '#000000', #-10.5
        '#000000', #-10
        '#000000', #-9.5
        '#000000', #-9
        '#000000', #-8.5
        '#000000', #-8
        '#000000', #-7.5
        '#000000', #-7
        '#000000', #-6.5
        '#000000', #-6
        '#000000', #-5.5
        '#000000', #-5
        '#000000', #-4.5
        '#000000', #-4
        '#000000', #-3.5
        '#000000', #-3
        '#000000', #-2.5
        '#000000', #-2
        '#000000', #-1.5
        '#000000', #-1
        '#000000', #-0.5
        '#000000', #0
        '#000000', #0.5
        '#000000', #1
        '#000000', #1.5
        '#000000', #2
        '#000000', #2.5
        '#000000', #3
        '#000000', #3.5
        '#000000', #4
        '#000000', #4.5
        '#000000', #5
        '#000000', #5.5
        '#000000', #6
        '#000000', #6.5
        '#000000', #7
        '#000000', #7.5
        '#000000', #8
        '#000000', #8.5
        '#000000', #9
        '#000000', #9.5
        '#000000', #10
        '#000000', #10.5
        '#a4a4ff', #11
        '#a1a1fc', #11.5
        '#9e9ef9', #12
        '#9a9af6', #12.5
        '#9797f2', #13
        '#9494ef', #13.5
        '#9191ec', #14
        '#8e8ee9', #14.5
        '#8a8ae6', #15
        '#8787e3', #15.5
        '#8484e0', #16
        '#8181dc', #16.5
        '#7e7ed9', #17
        '#7a7ad6', #17.5
        '#7777d3', #18
        '#7474d0', #18.5
        '#7171cd', #19
        '#6e6ec9', #19.5
        '#6a6ac6', #20
        '#6767c3', #20.5
        '#4080ff', #21
        '#3e7df9', #21.5
        '#3d7af2', #22
        '#3b76ec', #22.5
        '#3a73e6', #23
        '#3870df', #23.5
        '#366dd9', #24
        '#356ad3', #24.5
        '#3366cc', #25
        '#3263c6', #25.5
        '#3060c0', #26
        '#2e5db9', #26.5
        '#2d5ab3', #27
        '#2b56ac', #27.5
        '#2a53a6', #28
        '#2850a0', #28.5
        '#264d99', #29
        '#254a93', #29.5
        '#23468d', #30
        '#224386', #30.5
        '#204080', #31
        '#00f900', #31.5
        '#00f200', #32
        '#00ec00', #32.5
        '#00e600', #33
        '#00df00', #33.5
        '#00d900', #34
        '#00d300', #34.5
        '#00cc00', #35
        '#00c600', #35.5
        '#00c000', #36
        '#00b900', #36.5
        '#00b300', #37
        '#00ac00', #37.5
        '#00a600', #38
        '#00a000', #38.5
        '#009900', #39
        '#009300', #39.5
        '#008d00', #40
        '#008600', #40.5
        '#008000', #41
        '#fff900', #41.5
        '#fff200', #42
        '#ffec00', #42.5
        '#ffe600', #43
        '#ffdf00', #43.5
        '#ffd900', #44
        '#ffd300', #44.5
        '#ffcc00', #45
        '#ffc600', #45.5
        '#ffc000', #46
        '#ffb900', #46.5
        '#ffb300', #47
        '#ffac00', #47.5
        '#ffa600', #48
        '#ffa000', #48.0
        '#ff9900', #49
        '#ff9300', #49.5
        '#ff8d00', #50
        '#ff8600', #50.5
        '#ff0000', #51
        '#fa0000', #51.5
        '#f50000', #52
        '#f10000', #52.5
        '#ec0000', #53
        '#e70000', #53.5
        '#e30000', #54
        '#de0000', #54.5
        '#d90000', #55
        '#d40000', #55.5
        '#cf0000', #56
        '#cb0000', #56.5
        '#c60000', #57
        '#c10000', #57.5
        '#bd0000', #58
        '#b80000', #58.5
        '#b30000', #59
        '#ae0000', #59.5
        '#aa0000', #60
        '#a50000', #60.5
        '#ff00ff', #61
        '#f900f9', #61.5
        '#f200f2', #62
        '#ec00ec', #62.5
        '#e600e6', #63
        '#df00df', #63.5
        '#d900d9', #64
        '#d300d3', #64.5
        '#cc00cc', #65
        '#c600c6', #65.5
        '#c000c0', #66
        '#b900b9', #66.5
        '#b300b3', #67
        '#ac00ac', #67.5
        '#a600a6', #68
        '#a000a0', #68.5
        '#990099', #69
        '#930093', #69.5
        '#8d008d', #70
        '#860086', #70.5
        '#ffffff', #71
        '#f9f9f9', #71.5
        '#f2f2f2', #72
        '#e6e6e6', #73
        '#dfdfdf', #73.5
        '#d9d9d9', #74
        '#d3d3d3', #74.5
        '#cccccc', #75
        '#c6c6c6', #75.5
        '#c0c0c0', #76
        '#b9b9b9', #76.5
        '#b3b3b3', #77
        '#acacac', #77.5
        '#a6a6a6', #78
        '#a0a0a0', #78.5
        '#999999', #79
        '#939393', #79.5
        '#8d8d8d', #80
        '#868686', #80.5
        '#808080', #81
        '#808080', #81.5
        '#808080', #82
        '#808080', #82.5
        '#808080', #83
        '#808080', #83.5
        '#808080', #84
        '#808080', #84.5
        '#808080', #85
        '#808080', #85.5
        '#808080', #86
        '#808080', #86.5
        '#808080', #87
        '#808080', #87.5
        '#808080', #88
        '#808080', #88.5
        '#808080', #89
        '#808080', #89.5
        '#808080', #90
        '#808080', #90.5
        '#808080', #91
        '#808080', #91.5
        '#808080', #92
        '#808080', #92.5
        '#808080', #93
        '#808080', #93.5
        '#808080', #94
        '#808080', #94.5
        '#808080', #95
        ]
        
        
    return mpl.colors.ListedColormap(nws_reflectivity_colors)

new_cmap = IS_radar_colormap()

In [6]:
#Defintion to show image during labeling data.
def show_image(row_num):
    
    #Find the datafile corresponding to the current row in the pickle file
    data = glob.glob(f"/home/scratch/jcorner1/MODE_data/{pkl_data.iloc[[row_num]]['uniq_id'].values[0][4:12]}0000/mrms*")
    ds = xr.open_dataset(data[0])
    
    #subset parts of the data to be used in plotting
    rad = ds.mrms_lcref.values
    lat = ds.lat.values
    lon = ds.lon.values
    bounds = pkl_data.iloc[[row_num]][['xmin', 'ymin', 'xmax', 'ymax']].values[0]
    
    #Plot the data
    fig = plt.figure()
    fig.set_size_inches(16,12)
    ax1 = fig.add_subplot(111, projection = cartopy.crs.LambertConformal())
    plt.title(pkl_data.iloc[[row_num]]['uniq_id'].values[0])
    
    #Normalize the data based on the colormap
    norm = colors.Normalize(vmin=-32.5, vmax=95)

    #Create the image
    ax1.pcolormesh(lon[bounds[0]:bounds[2]+1], lat[bounds[1]:bounds[3]+1], rad[bounds[1]:bounds[3]+1, bounds[0]:bounds[2]+1], transform=cartopy.crs.PlateCarree(), shading='auto', cmap= new_cmap, norm=norm)
    return plt.show()

In [7]:
#Create a button to allow for selection of the convective mode for different storm objects.
#Defintion is executed when the button is pressed.
def on_button_clicked(b):
    
    #create list and variables for use within the defintion.
    #using globals is important with variables used as a counting function within a defintion. 
    lst = ['Supercell', 'Isolated Cell', 'Cluster of Cells', 'Supercell', 'Linear', 'Disorganized']
    global row
    global total
    
    #Open the text file to be written to.
    with open('storm_mode.txt', 'a') as f:
        
        
        #Logic statement if a convective mode is selected.
        if drop.value in lst:
            
            #This statement is needed for button to work when pressed
            with output:
                row += 1
                total += 1
                
                #Resets the output
                clear_output(wait=True)
                
                #plot the new data
                show_image(row-1)
                #print(pkl_data.iloc[[row-1]])
                
                print(f'Button clicked. {drop.value} selected! {total} modes picked.')
                f.write(f"{row-2},{pkl_data.iloc[row-2]['uniq_id']},{pkl_data.iloc[row-2]['area']},{pkl_data.iloc[row-2]['convex_area']},{pkl_data.iloc[row-2]['eccentricity']},{pkl_data.iloc[row-2]['mean_intensity']}"
                        f",{pkl_data.iloc[row-2]['max_intensity']},{pkl_data.iloc[row-2]['intensity_variance']},{pkl_data.iloc[row-2]['major_axis_length']},{pkl_data.iloc[row-2]['minor_axis_length']},"
                        f"{pkl_data.iloc[row-2]['solidity']},{pkl_data.iloc[row-2]['convection_area']},{pkl_data.iloc[row-2]['intense_area']},{drop.value}\n")
        
        #Logic statement if a convective mode isn't selected.
        else:
            
            #This statement is needed for button to work when pressed
            with output:
                
                #Resets the output
                clear_output(wait=True)
                
                print('skipped')
                row += 1
                
                #print(pkl_data.iloc[[row-1]])
                show_image(row-1)           

## Labeling Data

In [8]:
#Create the widgets for the buttons and the output.
#Output widget is needed to print out information.
button = widgets.Button(description="Next")
output = widgets.Output()

#Display the buttons and the output
display(drop)
display(button, output)

#When the button is clicked the the defintion is executed. 
button.on_click(on_button_clicked)


Dropdown(description='Storm Mode:', index=5, options=('Isolated Cell', 'Cluster of Cells', 'Supercell', 'Linea…

Button(description='Next', style=ButtonStyle())

Output()